In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [8]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/P4/Crude Oil Prices Daily.csv')
df = df.dropna()
df.isnull().sum()



Date             0
Closing Value    0
dtype: int64

In [9]:
# Preprocess data
scaler = MinMaxScaler()
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df['Scaled_Price'] = scaler.fit_transform(df[['Closing Value']])



In [10]:
# Create sequences for LSTM
sequence_length = 30
sequences = []
prices = df['Scaled_Price'].to_numpy()

for i in range(len(prices) - sequence_length):
    sequences.append(prices[i:i+sequence_length+1])

In [11]:
# Convert to NumPy array
sequences = np.array(sequences)

In [12]:
# Split data into features and target
X, y = sequences[:, :-1], sequences[:, -1]

In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Reshape data for LSTM (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [15]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [16]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
205/205 [==============================] - 6s 17ms/step - loss: 0.0039 - val_loss: 2.8583e-04
Epoch 2/50
205/205 [==============================] - 4s 21ms/step - loss: 2.8832e-04 - val_loss: 2.4222e-04
Epoch 3/50
205/205 [==============================] - 3s 14ms/step - loss: 2.6180e-04 - val_loss: 2.2120e-04
Epoch 4/50
205/205 [==============================] - 4s 18ms/step - loss: 2.4194e-04 - val_loss: 2.0200e-04
Epoch 5/50
205/205 [==============================] - 3s 14ms/step - loss: 2.1503e-04 - val_loss: 1.9200e-04
Epoch 6/50
205/205 [==============================] - 5s 23ms/step - loss: 2.0464e-04 - val_loss: 1.8353e-04
Epoch 7/50
205/205 [==============================] - 3s 15ms/step - loss: 1.9311e-04 - val_loss: 1.7636e-04
Epoch 8/50
205/205 [==============================] - 3s 14ms/step - loss: 1.8189e-04 - val_loss: 1.6259e-04
Epoch 9/50
205/205 [==============================] - 3s 14ms/step - loss: 1.7177e-04 - val_loss: 1.6203e-04
Epoch 10/50
205/205 [==

In [17]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Model Loss on Test Data: {loss}")

52/52 [==============================] - 0s 6ms/step - loss: 8.3643e-05
Model Loss on Test Data: 8.364320819964632e-05


In [18]:
# Save the model
model.save("crude_oil_price_lstm_model")